<center><img src = "practicals/Images/ATU_Logo.jpg"></center>

<h2 style="color: #001a79;">Author: Sam Tracey</h2>
<h2 style="color: #001a79;">Assessment</h2>
<h2 style="color: #001a79;">Time Series Anomaly Detection with Keras.</h2>
<h2 style="color: #001a79;">https://keras.io/examples/timeseries/timeseries_anomaly_detection/</h2>
<hr style="border-top: 1px solid #001a79;" />

